# Les 5: Afgeleiden en de Kettingregel

**Mathematical Foundations - IT & Artificial Intelligence**

---

## 5.0 Recap en Motivatie

In de vorige lessen hebben we geleerd hoe data door een neuraal netwerk stroomt. We kunnen nu een forward pass uitvoeren: gegeven een input, berekenen we de output via matrixvermenigvuldigingen en activatiefuncties. Maar tot nu toe waren onze netwerken "dom" - ze hadden random gewichten en maakten willekeurige voorspellingen.

De grote vraag is: hoe kan een netwerk leren? Hoe passen we de gewichten aan zodat de voorspellingen beter worden?

Het kernidee is verrassend eenvoudig. We meten hoe "fout" het netwerk is via een loss functie. Dan vragen we: als ik een bepaalde weight een klein beetje verander, wordt de fout dan groter of kleiner? En hoeveel? Als we dit voor alle weights weten, kunnen we ze allemaal een beetje aanpassen in de richting die de fout verkleint.

Dit "hoeveel verandert de fout als ik de weight verander" is precies wat een afgeleide meet. In deze les leren we afgeleiden: de wiskundige taal van verandering.

## 5.1 Leerdoelen

Na deze les kun je de afgeleide intuïtief begrijpen als de maat voor verandering. Je kunt afgeleiden numeriek benaderen en analytisch berekenen voor basisregels. Je beheerst de kettingregel voor samengestelde functies. Je kunt partiële afgeleiden berekenen en de gradiënt vormen. Je begrijpt waarom dit essentieel is voor het trainen van neurale netwerken.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)
print("Libraries geladen!")

## 5.2 De Afgeleide: Intuïtie

### Wat meet een afgeleide?

Stel je voor dat je met de auto rijdt en je kijkt naar de snelheidsmeter. De snelheid vertelt je hoe snel je positie verandert. Als je 60 km/u rijdt, dan verander je elke seconde ongeveer 16.7 meter van positie.

De snelheid is de afgeleide van positie naar tijd. Meer algemeen: de afgeleide van een functie f(x) op een punt x vertelt je hoe snel f verandert als x verandert. Het is de helling van de functie op dat punt.

Wiskundig benaderen we dit door te kijken naar het verschil f(x+h) - f(x) voor een kleine stap h, gedeeld door h:

f'(x) ≈ (f(x+h) - f(x)) / h

Als h steeds kleiner wordt, nadert deze benadering de echte afgeleide.

In [ ]:
# Voorbeeld: f(x) = x²
def f(x):
    return x ** 2

# Numerieke afgeleide
def numerical_derivative(f, x, h=1e-5):
    return (f(x + h) - f(x)) / h

# Test op x = 3
x = 3
print(f"f(x) = x² op x = {x}")
print(f"f({x}) = {f(x)}")
print()

# Numerieke benadering met verschillende h
print("Numerieke afgeleide met verschillende stapgroottes:")
for h in [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]:
    deriv = numerical_derivative(f, x, h)
    print(f"  h = {h:8.5f}: f'({x}) ≈ {deriv:.6f}")

print()
print(f"Analytische afgeleide: f'(x) = 2x, dus f'({x}) = {2*x}")

In [ ]:
# Visualisatie: de afgeleide als helling van de raaklijn
x_range = np.linspace(0, 5, 100)
y_range = f(x_range)

# Punt waar we de afgeleide berekenen
x0 = 2
y0 = f(x0)
slope = 2 * x0  # Afgeleide van x² is 2x

# Raaklijn: y - y0 = slope * (x - x0)
tangent_y = slope * (x_range - x0) + y0

plt.figure(figsize=(10, 6))
plt.plot(x_range, y_range, 'b-', linewidth=2, label='f(x) = x²')
plt.plot(x_range, tangent_y, 'r--', linewidth=2, label=f'Raaklijn (helling = {slope})')
plt.plot(x0, y0, 'go', markersize=10, label=f'Punt ({x0}, {y0})')

plt.xlim(0, 5)
plt.ylim(0, 20)
plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.title(f'De afgeleide is de helling van de raaklijn\nf\'({x0}) = {slope}', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print(f"Op x = {x0}: de helling is {slope}, dus als x met 1 toeneemt,")
print(f"neemt f(x) met ongeveer {slope} toe.")

In [ ]:
# De afgeleide op verschillende punten
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

x_range = np.linspace(-3, 3, 100)

for ax, x0 in zip(axes, [-1, 0, 2]):
    y_range = x_range ** 2
    y0 = x0 ** 2
    slope = 2 * x0
    tangent_y = slope * (x_range - x0) + y0
    
    ax.plot(x_range, y_range, 'b-', linewidth=2)
    ax.plot(x_range, tangent_y, 'r--', linewidth=2)
    ax.plot(x0, y0, 'go', markersize=10)
    
    ax.set_xlim(-3, 3)
    ax.set_ylim(-2, 8)
    ax.set_xlabel('x')
    ax.set_title(f'x = {x0}: helling = {slope}')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)

plt.suptitle('f(x) = x²: de afgeleide f\'(x) = 2x op verschillende punten', fontsize=14)
plt.tight_layout()
plt.show()

## 5.3 Basisregels voor Afgeleiden

In plaats van elke afgeleide numeriek te berekenen, kunnen we analytische regels gebruiken. Deze regels zijn sneller en nauwkeuriger. De belangrijkste regels zijn:

**Constante regel:** De afgeleide van een constante is nul.
d/dx[c] = 0

**Machtregel:** De afgeleide van xⁿ is n·xⁿ⁻¹.
d/dx[xⁿ] = n·xⁿ⁻¹

**Somregel:** De afgeleide van een som is de som van de afgeleiden.
d/dx[f(x) + g(x)] = f'(x) + g'(x)

**Constante factor:** Een constante factor blijft behouden.
d/dx[c·f(x)] = c·f'(x)

**Exponentiële functie:** De afgeleide van eˣ is eˣ zelf.
d/dx[eˣ] = eˣ

In [ ]:
# Voorbeeld: f(x) = 3x³ + 2x² - 5x + 7
def f(x):
    return 3*x**3 + 2*x**2 - 5*x + 7

# Analytische afgeleide: f'(x) = 9x² + 4x - 5
def f_prime(x):
    return 9*x**2 + 4*x - 5

# Vergelijk numeriek en analytisch
print("f(x) = 3x³ + 2x² - 5x + 7")
print("f'(x) = 9x² + 4x - 5")
print()

test_points = [-2, -1, 0, 1, 2, 3]
print("Vergelijking numeriek vs analytisch:")
print(f"{'x':>4} | {'Numeriek':>12} | {'Analytisch':>12} | {'Verschil':>12}")
print("-" * 50)
for x in test_points:
    num = numerical_derivative(f, x)
    ana = f_prime(x)
    diff = abs(num - ana)
    print(f"{x:>4} | {num:>12.6f} | {ana:>12.6f} | {diff:>12.2e}")

In [ ]:
# De exponentiële functie: uniek!
x_range = np.linspace(-2, 2, 100)

plt.figure(figsize=(10, 6))
plt.plot(x_range, np.exp(x_range), 'b-', linewidth=2, label='f(x) = eˣ')

# De afgeleide is dezelfde functie!
plt.plot(x_range, np.exp(x_range), 'r--', linewidth=2, label="f'(x) = eˣ (zelfde!)")

plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('De exponentiële functie eˣ is zijn eigen afgeleide!', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("Dit is waarom e zo speciaal is in wiskunde en natuurkunde.")
print("Het is de enige functie (op constante factor na) die gelijk is aan zijn afgeleide.")

## 5.4 De Kettingregel

### Het probleem

Wat als we de afgeleide willen van een samengestelde functie? Bijvoorbeeld f(x) = (3x + 2)². We kunnen dit uitwerken tot 9x² + 12x + 4 en dan differentiëren, maar voor complexere functies is dat niet praktisch.

### De kettingregel

De kettingregel zegt: als h(x) = f(g(x)), dan is h'(x) = f'(g(x)) · g'(x).

In woorden: de afgeleide van de buitenste functie (geëvalueerd op de binnenste), vermenigvuldigd met de afgeleide van de binnenste functie.

### Waarom is dit cruciaal voor neurale netwerken?

Een neuraal netwerk is één grote samengestelde functie! De output is iets als: softmax(W₂ · ReLU(W₁ · x + b₁) + b₂). Om de afgeleide naar W₁ te berekenen, moeten we de kettingregel meerdere keren toepassen. Dit is precies wat backpropagation doet.

In [ ]:
# Voorbeeld: h(x) = (3x + 2)²
# g(x) = 3x + 2 (binnenste)
# f(u) = u² (buitenste)
# h(x) = f(g(x))

def g(x):
    return 3*x + 2

def f(u):
    return u**2

def h(x):
    return f(g(x))  # = (3x + 2)²

# Afgeleiden
def g_prime(x):
    return 3  # d/dx[3x + 2] = 3

def f_prime(u):
    return 2*u  # d/du[u²] = 2u

def h_prime(x):
    # Kettingregel: f'(g(x)) * g'(x)
    return f_prime(g(x)) * g_prime(x)

# Test
print("h(x) = (3x + 2)²")
print("g(x) = 3x + 2  →  g'(x) = 3")
print("f(u) = u²      →  f'(u) = 2u")
print()
print("Kettingregel: h'(x) = f'(g(x)) · g'(x) = 2(3x+2) · 3 = 6(3x+2)")
print()

x = 2
print(f"Op x = {x}:")
print(f"  g({x}) = {g(x)}")
print(f"  h({x}) = {h(x)}")
print(f"  h'({x}) analytisch = 6(3·{x}+2) = 6·{g(x)} = {6*g(x)}")
print(f"  h'({x}) via kettingregel = {h_prime(x)}")
print(f"  h'({x}) numeriek = {numerical_derivative(h, x):.4f}")

In [ ]:
# Meer complex voorbeeld: sigmoid functie
# σ(x) = 1 / (1 + e^(-x))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Afgeleide van sigmoid (kan je afleiden met de kettingregel)
# σ'(x) = σ(x) · (1 - σ(x))
def sigmoid_prime(x):
    s = sigmoid(x)
    return s * (1 - s)

# Visualisatie
x_range = np.linspace(-6, 6, 100)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(x_range, sigmoid(x_range), 'b-', linewidth=2)
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('σ(x)', fontsize=12)
axes[0].set_title('Sigmoid functie: σ(x) = 1/(1+e⁻ˣ)', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)

axes[1].plot(x_range, sigmoid_prime(x_range), 'r-', linewidth=2)
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel("σ'(x)", fontsize=12)
axes[1].set_title("Afgeleide: σ'(x) = σ(x)·(1-σ(x))", fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Verifieer numeriek
x = 1
print(f"\nVerificatie op x = {x}:")
print(f"  σ'({x}) analytisch = {sigmoid_prime(x):.6f}")
print(f"  σ'({x}) numeriek   = {numerical_derivative(sigmoid, x):.6f}")

In [ ]:
# ReLU en zijn afgeleide
def relu(x):
    return np.maximum(0, x)

def relu_prime(x):
    return np.where(x > 0, 1, 0)  # 1 als x > 0, anders 0

x_range = np.linspace(-3, 3, 100)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(x_range, relu(x_range), 'b-', linewidth=2)
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('ReLU(x)', fontsize=12)
axes[0].set_title('ReLU: max(0, x)', fontsize=12)
axes[0].grid(True, alpha=0.3)

axes[1].plot(x_range, relu_prime(x_range), 'r-', linewidth=2)
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel("ReLU'(x)", fontsize=12)
axes[1].set_title("Afgeleide: 1 als x > 0, anders 0", fontsize=12)
axes[1].set_ylim(-0.5, 1.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("De ReLU afgeleide is 'aan' (1) of 'uit' (0).")
print("Dit maakt ReLU computationeel efficiënt.")

## 5.5 Partiële Afgeleiden

### Functies met meerdere variabelen

Tot nu toe hadden we functies van één variabele: f(x). Maar in neurale netwerken hebben we te maken met functies van vele variabelen: de loss L hangt af van alle weights w₁, w₂, ..., wₙ.

Een partiële afgeleide meet hoe de functie verandert als we één variabele veranderen terwijl we alle andere constant houden.

### Notatie

De partiële afgeleide van f naar x noteren we als ∂f/∂x (in plaats van df/dx). Het symbool ∂ ("del" of "partial") geeft aan dat er meerdere variabelen zijn.

### De gradiënt

De gradiënt van f is de vector van alle partiële afgeleiden:

∇f = [∂f/∂x₁, ∂f/∂x₂, ..., ∂f/∂xₙ]

De gradiënt wijst in de richting van de steilste stijging van de functie.

In [ ]:
# Voorbeeld: f(x, y) = x² + xy + y²
def f(x, y):
    return x**2 + x*y + y**2

# Partiële afgeleiden (analytisch)
# ∂f/∂x = 2x + y
# ∂f/∂y = x + 2y

def df_dx(x, y):
    return 2*x + y

def df_dy(x, y):
    return x + 2*y

def gradient(x, y):
    return np.array([df_dx(x, y), df_dy(x, y)])

# Numerieke partiële afgeleiden
def numerical_partial_x(f, x, y, h=1e-5):
    return (f(x + h, y) - f(x, y)) / h

def numerical_partial_y(f, x, y, h=1e-5):
    return (f(x, y + h) - f(x, y)) / h

# Test
x0, y0 = 2, 3
print(f"f(x, y) = x² + xy + y²")
print(f"\nOp punt ({x0}, {y0}):")
print(f"  f({x0}, {y0}) = {f(x0, y0)}")
print()
print(f"  ∂f/∂x analytisch = 2x + y = 2·{x0} + {y0} = {df_dx(x0, y0)}")
print(f"  ∂f/∂x numeriek   = {numerical_partial_x(f, x0, y0):.4f}")
print()
print(f"  ∂f/∂y analytisch = x + 2y = {x0} + 2·{y0} = {df_dy(x0, y0)}")
print(f"  ∂f/∂y numeriek   = {numerical_partial_y(f, x0, y0):.4f}")
print()
print(f"  Gradiënt: ∇f = {gradient(x0, y0)}")

In [ ]:
# Visualisatie: gradiënt als pijlen op een contourplot
x_range = np.linspace(-3, 3, 50)
y_range = np.linspace(-3, 3, 50)
X, Y = np.meshgrid(x_range, y_range)
Z = f(X, Y)

# Gradiënt op een grove grid
x_arrows = np.linspace(-2.5, 2.5, 8)
y_arrows = np.linspace(-2.5, 2.5, 8)
X_arr, Y_arr = np.meshgrid(x_arrows, y_arrows)

# Gradiënt componenten
U = df_dx(X_arr, Y_arr)
V = df_dy(X_arr, Y_arr)

plt.figure(figsize=(10, 8))
plt.contour(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(label='f(x, y)')
plt.quiver(X_arr, Y_arr, U, V, color='red', alpha=0.7)
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('f(x, y) = x² + xy + y²\nRode pijlen: gradiënt (richting van steilste stijging)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

print("De gradiënt wijst altijd loodrecht op de contourlijnen,")
print("in de richting waar de functie het snelst stijgt.")

## 5.6 Toepassing: Afgeleide van een Simpel Neuron

Laten we alles samenvoegen en de afgeleiden berekenen voor een enkel neuron. Een neuron berekent:

y = σ(wx + b)

waarbij σ de sigmoid activatiefunctie is. We willen weten hoe de output y verandert als we w of b aanpassen. Dit zijn de partiële afgeleiden ∂y/∂w en ∂y/∂b.

In [ ]:
# Neuron: y = σ(wx + b)

def neuron(x, w, b):
    """Bereken de output van een neuron."""
    z = w * x + b  # Lineaire combinatie
    y = sigmoid(z)  # Activatie
    return y

# Afgeleiden met de kettingregel
# y = σ(z) waar z = wx + b
# ∂y/∂w = ∂y/∂z · ∂z/∂w = σ'(z) · x
# ∂y/∂b = ∂y/∂z · ∂z/∂b = σ'(z) · 1

def neuron_gradients(x, w, b):
    """Bereken de gradiënten van een neuron."""
    z = w * x + b
    y = sigmoid(z)
    
    # σ'(z) = σ(z)(1 - σ(z)) = y(1 - y)
    dy_dz = y * (1 - y)
    
    # Kettingregel
    dy_dw = dy_dz * x  # ∂z/∂w = x
    dy_db = dy_dz * 1  # ∂z/∂b = 1
    
    return dy_dw, dy_db

# Test
x = 2.0
w = 0.5
b = -0.3

y = neuron(x, w, b)
dy_dw, dy_db = neuron_gradients(x, w, b)

print(f"Neuron: y = σ(wx + b)")
print(f"\nInput: x = {x}, w = {w}, b = {b}")
print(f"\nz = wx + b = {w}·{x} + {b} = {w*x + b}")
print(f"y = σ(z) = {y:.6f}")
print()
print(f"Gradiënten:")
print(f"  ∂y/∂w = {dy_dw:.6f}")
print(f"  ∂y/∂b = {dy_db:.6f}")

In [ ]:
# Verifieer numeriek
h = 1e-5

# Numerieke ∂y/∂w
dy_dw_num = (neuron(x, w + h, b) - neuron(x, w, b)) / h

# Numerieke ∂y/∂b
dy_db_num = (neuron(x, w, b + h) - neuron(x, w, b)) / h

print("Verificatie met numerieke afgeleiden:")
print(f"  ∂y/∂w analytisch: {dy_dw:.6f}")
print(f"  ∂y/∂w numeriek:   {dy_dw_num:.6f}")
print()
print(f"  ∂y/∂b analytisch: {dy_db:.6f}")
print(f"  ∂y/∂b numeriek:   {dy_db_num:.6f}")

In [ ]:
# Visualiseer hoe de output verandert met w en b
x = 2.0  # Vaste input

w_range = np.linspace(-2, 2, 50)
b_range = np.linspace(-2, 2, 50)
W, B = np.meshgrid(w_range, b_range)

# Bereken output voor alle combinaties
Y = sigmoid(W * x + B)

fig = plt.figure(figsize=(14, 5))

# 3D surface plot
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(W, B, Y, cmap='viridis', alpha=0.8)
ax1.set_xlabel('w')
ax1.set_ylabel('b')
ax1.set_zlabel('y')
ax1.set_title(f'Output y = σ(wx + b) met x = {x}')

# Contour plot met gradiënten
ax2 = fig.add_subplot(122)
contour = ax2.contour(W, B, Y, levels=15, cmap='viridis')
ax2.clabel(contour, inline=True, fontsize=8)

# Bereken gradiënten op grove grid
w_arr = np.linspace(-1.5, 1.5, 6)
b_arr = np.linspace(-1.5, 1.5, 6)
W_arr, B_arr = np.meshgrid(w_arr, b_arr)

Z = W_arr * x + B_arr
Y_arr = sigmoid(Z)
dY_dZ = Y_arr * (1 - Y_arr)
dY_dW = dY_dZ * x
dY_dB = dY_dZ * 1

ax2.quiver(W_arr, B_arr, dY_dW, dY_dB, color='red', alpha=0.7)
ax2.set_xlabel('w')
ax2.set_ylabel('b')
ax2.set_title('Contour met gradiënt pijlen')

plt.tight_layout()
plt.show()

## 5.7 Samenvatting en Vooruitblik

### Kernconcepten

In deze les hebben we geleerd dat afgeleiden meten hoe snel een functie verandert. We kunnen ze numeriek benaderen of analytisch berekenen met regels zoals de machtregel en de kettingregel.

De kettingregel is cruciaal voor samengestelde functies: (f ∘ g)'(x) = f'(g(x)) · g'(x). Dit is precies wat we nodig hebben voor neurale netwerken, die uit meerdere samengestelde lagen bestaan.

Partiële afgeleiden meten verandering in functies met meerdere variabelen. De gradiënt is de vector van alle partiële afgeleiden en wijst in de richting van steilste stijging.

### Link naar neurale netwerken

We hebben gezien hoe we de afgeleiden van een enkel neuron berekenen: hoe de output verandert als we de weight of bias aanpassen. In een echt netwerk moeten we dit voor duizenden parameters doen, en de kettingregel door meerdere lagen toepassen.

### Volgende les

In les 6 leren we gradient descent: hoe we de gradiënt gebruiken om parameters te optimaliseren. We gaan de loss minimaliseren door steeds kleine stapjes in de richting van de negatieve gradiënt te nemen.

### Checklist

Controleer of je het volgende begrijpt:

1. Wat meet een afgeleide?

2. Hoe bereken je de afgeleide van xⁿ?

3. Wat zegt de kettingregel?

4. Wat is een partiële afgeleide?

5. Wat is de gradiënt en welke richting wijst deze?

Als je deze vragen kunt beantwoorden, ben je klaar voor les 6!

---

**Mathematical Foundations** | Les 5 van 12 | IT & Artificial Intelligence

---